In [118]:
import requests
import json
import pandas as pd
from datetime import datetime
import statistics
from IPython.display import HTML, display
from wimbledon.vis.Visualise import DataHandlers

In [2]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


GitHub emojis
=======

Project issues that members of the REG team have selected, "LAUGH" are the ones they would most like to be assigned to, "THUMBS_UP" they would also be happy with, "THUMBS_DOWN" they would not be happy with.

In [166]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '✔️',
          'LAUGH': '😀'}

In [4]:
fc = DataHandlers.Forecast()  # get data from forecast

Enpoint: whoami | Time: 0.5059723990125349 
Enpoint: people/475627 | Time: 0.48271679900062736 

AUTHENTICATED USER:
Ed Chalstrey echalstrey@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.5350835980061674 
PROJECTS
Enpoint: projects | Time: 0.7616336270002648 
ROLES
Enpoint: roles | Time: 0.6055478719936218 
PEOPLE
Enpoint: people | Time: 0.5945824310038006 
PLACEHOLDERS
Enpoint: placeholders | Time: 0.48869862699939404 
MILESTONES
Enpoint: milestones | Time: 0.4685551090078661 
ASSIGNMENTS
Enpoint: assignments | Time: 0.7180030739982612 
DONE! (5.2s)


REG team
=====

We need a list of the team members full names from Forecast to compare with their names on GitHub. They match most of the time except when there is no GitHub name. Mapping below covers the rest of the team. We need to update this list if someone new is added to Forecast who doesn't have a matching GitHub name.

In [141]:
names = list(fc.people.full_name)

In [82]:
gid_mapping = {  # People without their full names on github.
 'myyong': 'May Yong',
 'nbarlowATI': 'Nick Barlow',
 'thobson88': 'Timothy Hobson',
 'miguelmorin': 'Miguel Morin',
 'OscartGiles': 'Oscar Giles',
 'AshwiniKV': 'Ashwini Venkatasubramaniam',
}

Find a team member's availability for a date range
---

In [203]:
def get_person_availability(name, peopledf, start_date, end_date):
    try:
        person_id = fc.get_person_id(name)
    except:
        return 0.0
    peopledf = peopledf[(peopledf.index >= start_date) & (peopledf.index <= end_date)]
    availability_range = peopledf[person_id]
#     print(availability_range)
    average_availability = statistics.mean(availability_range)
    return round(average_availability, 2)

In [197]:
peopledf = 1 - fc.people_totals.resample('MS').mean()  # pandas df for team members availability
start_date = datetime(2019, 10, 1)
end_date = datetime(2020, 10, 1)

get_person_availability("Ed Chalstrey", peopledf, start_date, end_date)

# peopledf.columns=[fc.get_person_name(idx) for idx in peopledf.columns]
# peopledf["James Robinson"]

0.77

Create queries with GraphQL to get the emojis for each GitHub issue
===

In [144]:
issues = fc.projects["GitHub"].dropna().values  # Get list of GitHub issues for projects

In [43]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

Check if a GitHub issue has emojis, based on the result of GraphQL query
----

In [45]:
def query_result_contains_emojis(query_result):
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        if len(reaction['users']['edges']) > 0:
            return True
    return False

Generate a HTML table displaying the results of emoji rating and availability of each team member for each project
---

In [204]:
def tabulate_emojis(query_result, names, gid_mapping, peopledf, start_date, end_date):
    data = [
        ["Person"],
        ["Rating"],
        ["Availability"]
    ]
    def add_to_data(laugh=False):
        for reaction in query_result['data']['repository']['issue']['reactionGroups']:
            if (reaction['content'] == 'LAUGH' and laugh) or (reaction['content'] != 'LAUGH' and not laugh):
                for edge in reaction['users']['edges']:
                    if edge['node']['name'] in names:
                        name = edge['node']['name']
                    elif edge['node']['login'] in gid_mapping:
                        name = gid_mapping[edge['node']['login']]
                        
                    person_availability = get_person_availability(name, peopledf, start_date, end_date)
                    
                    if person_availability >= 0.1:
                        
                        data[0].append(name)

                        data[1].append(emojis[reaction['content']])

                        data[2].append(person_availability)
                
    add_to_data(laugh=True)
    add_to_data(laugh=False)

    display(HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
       )
    ))


Get the projects for which resources (team members) are required and display availability and preference for each team member who has with reacted with an emoji on the GitHub issue
----

In [200]:
resreqdf = fc.project_resourcereq.resample('MS').mean() # grouped by month and mean taken
resreqdf

id,1684536,1684539,1723251,1723252,1723253,1723254,1723255,1723256,1723257,1723258,...,2061037,2061048,2061076,2061084,2063544,2063583,2065747,2065758,2065776,2150325
2016-11-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2016-12-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-02-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-03-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-04-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-05-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-06-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-07-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-08-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0


In [206]:
for project_id in resreqdf:
    for resreq in resreqdf[project_id]:
        if resreq > 0:  # if at least one month in the dataframe has a resource requirement of more than 0 FTE
            issue_num = fc.projects.loc[project_id, "GitHub"]
            if issue_num in issues:
                print(issue_num, fc.projects.loc[project_id, "name"],
                      "Start:", fc.projects.loc[project_id, "start_date"],
                      "End:", fc.projects.loc[project_id, "end_date"], 
                      round(resreq, 3), "FTE")
                
                modified_query = query.replace("X", str(issue_num))
                result = run_query(modified_query)  # Execute the query
                if query_result_contains_emojis(result):  # Currently only displaying those with at least 1 emoji
                    tabulate_emojis(result, names, gid_mapping, peopledf, fc.projects.loc[project_id, "start_date"], fc.projects.loc[project_id, "end_date"])
                else:
                    print("NO EMOJIS FOUND")
            break

96 Living With Machines Start: 2018-06-01 End: 2022-03-31 1.0 FTE


Person,Jack Roberts,Sarah Gibson,Kasra Hosseini,Nick Barlow,Camila Rangel Smith,Evelina Gabasova,Timothy Hobson,Radka Jersakova,James Robinson,Louise Bowler,Oscar Giles,Tomas Lazauskas,May Yong,Jim Madge
Rating,😀,😀,😀,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌,❌
Availability,0.49,0.5,0.44,0.55,0.61,0.29,0.29,0.64,0.5,0.49,0.69,0.57,0.58,0.58


311 AIDA-Lloyds Start: 2017-04-01 End: 2019-12-31 0.4 FTE


Person
Rating
Availability


183 Probabilistic FEM Start: 2018-12-03 End: 2020-02-29 0.5 FTE


...

Person,Ed Chalstrey,Sarah Gibson,James Robinson,Louise Bowler,Jim Madge,Eric Daub,Jack Roberts,Nick Barlow,Evelina Gabasova,Oscar Giles,Radka Jersakova,Roly Perera,May Yong,Ashwini Venkatasubramaniam,Kasra Hosseini,Miguel Morin,Timothy Hobson
Rating,😀,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌,❌,❌,❌,❌
Availability,0.96,0.21,0.2,0.46,0.5,0.21,0.5,0.29,0.21,0.71,0.62,0.92,0.37,0.75,0.25,0.12,0.21


332 Scaleable Monte Carlo Start: 2020-10-01 End: 2021-09-30 1.0 FTE


Person,James Robinson,Nick Barlow,Louise Bowler,Jim Madge,Eric Daub,Oscar Giles,Tomas Lazauskas,Timothy Hobson,May Yong,Sarah Gibson,Camila Rangel Smith,Evelina Gabasova,Kasra Hosseini,Radka Jersakova
Rating,😀,😀,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌,❌,❌,❌,❌
Availability,1.0,1.0,0.75,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,0.5,0.75,1.0


293 Wrattler 2019-2020 Start: 2019-04-01 End: 2020-03-31 1.0 FTE


Person,Sarah Gibson
Rating,😀
Availability,0.12
